In [2]:
from pymongo import MongoClient
import pprint

In [3]:
client = MongoClient()

In [4]:
db = client.radio

In [5]:
materials = db.material

In [6]:
variableSkelett = {
    "id": "S8",
    "textBefore": "",
    "textAfter": "",
    "keys": [
        ["frisch"],
        ["alt"]
    ],
    "kind": "oc",
    "value": None,
    "values": ["frisch", "alt"]
}

In [7]:
variablePneumonie0 = {
    "id": "Pn2",
    "textBefore": "",
    "textAfter": "",
    "keys": [
        ["OL", "Oberfeld", "Oberlappen"],
        ["UL", "Unterfeld", "Unterlappen"],
        ["ML", "Mittelfeld", "Mittellappen"]
    ],
    "kind": "mc",
    "values": [
        ["OL", False],
        ["UL", False],
        ["ML", False]
    ]
}

In [8]:
variablePneumonieOL = {
    "id": "Pn2",
    "textBefore": "",
    "textAfter": "",
    "keys": [
        ["OL", "Oberfeld", "Oberlappen"],
        ["UL", "Unterfeld", "Unterlappen"],
        ["ML", "Mittelfeld", "Mittellappen"]
    ],
    "kind": "mc",
    "values": [
        ["OL", True],
        ["UL", False],
        ["ML", False]
    ]
}

In [9]:
variablePneumonieUL = {
    "id": "Pn2",
    "textBefore": "",
    "textAfter": "",
    "keys": [
        ["OL", "Oberfeld", "Oberlappen"],
        ["UL", "Unterfeld", "Unterlappen"],
        ["ML", "Mittelfeld", "Mittellappen"]
    ],
    "kind": "mc",
    "values": [
        ["OL", False],
        ["UL", True],
        ["ML", False]
    ]
}

In [10]:
variablePneumonieML = {
    "id": "Pn2",
    "textBefore": "",
    "textAfter": "",
    "keys": [
        ["OL", "Oberfeld", "Oberlappen"],
        ["UL", "Unterfeld", "Unterlappen"],
        ["ML", "Mittelfeld", "Mittellappen"]
    ],
    "kind": "mc",
    "values": [
        ["OL", False],
        ["UL", False],
        ["ML", True]
    ]
}

In [17]:
oids = []
for material in materials.find({"judged": True}):
    oid = material.get("_id")
    oids.append(oid)

In [12]:
pprint.pprint(type(materials.find_one()))

<class 'dict'>


In [18]:
oids

[ObjectId('6319ff9f61d904cea90efafb'),
 ObjectId('6319ff9f61d904cea90efb07'),
 ObjectId('6319ffa061d904cea90efb92'),
 ObjectId('6319ffa061d904cea90efb98'),
 ObjectId('6319ffa061d904cea90efb9e'),
 ObjectId('6319ffa061d904cea90efba4'),
 ObjectId('6319ffa061d904cea90efba8'),
 ObjectId('6319ffa061d904cea90efbae'),
 ObjectId('631a0597daecbb0ad265e8ad'),
 ObjectId('631ae83c8a0ba52d17b5ac2f')]

In [28]:
for oid in oids:
    material = materials.find_one({"_id": oid})
    variables = material.get("deepDocTemplate").get("parts")[23].get("selectables")[11].get("variables")
    if len(variables) < 3:
        variables.append(variableSkelett)


    materials.replace_one({"_id": oid}, material)

{'__v': 0,
 '_id': '6319fc6861d904cea90ef9d4',
 'kind': 'deepDoc',
 'name': 'Radiolearn',
 'parts': [{'kind': 'category',
            'name': 'Aufnahmetyp',
            'optional': True,
            'selectables': [{'kind': 'group',
                             'name': 'ATG0',
                             'options': [{'keys': ['Thorax p.a.', 'Thorax pa'],
                                          'kind': 'option',
                                          'name': 'Thorax p.a.',
                                          'normal': False,
                                          'text': 'Projektionsradiographie: '
                                                  'Thorax in 1 Ebene ',
                                          'variables': []},
                                         {'keys': ['Thorax 2 Ebenen',
                                                   'Thorax in zwei Ebenen',
                                                   'Thorax zwei Ebenen',
                             

In [38]:
oid = oids[1]
material = materials.find_one({"_id": oid})
variables = material.get("deepDocTemplate").get("parts")[23].get("selectables")[11].get("variables")
if len(variables) < 3:
    variables.append(variableSkelett)

pneumoVariable = material.get("deepDocTemplate").get("parts")[16].get("selectables")[0].get("options")[1].get("variables")
pprint.pprint(material.get("deepDocTemplate").get("parts")[23])

{'kind': 'category',
 'name': 'Skelett',
 'optional': True,
 'selectables': [{'exclusions': ['Rest'],
                  'keys': ['keine Degenerationen',
                           'keine Degeneration',
                           'Skelett unauffällig'],
                  'kind': 'box',
                  'name': 'Skelett unauffällig',
                  'normal': True,
                  'text': 'Keine relevanten degenerativen Veränderungen.',
                  'value': True,
                  'variables': []},
                 {'enumeration': 'SZ0',
                  'exclusions': ['Skelett unauffällig'],
                  'keys': ['Gefügestörung'],
                  'kind': 'box',
                  'name': 'Gefügestörung',
                  'normal': False,
                  'text': 'des Gefüges %S8% [auf Höhe %S9%]',
                  'value': False,
                  'variables': [{'id': 'S8',
                                 'keys': [['gering'], ['mäßig'], ['deutlich']],
             